You should consider upgrading via the 'c:\Users\SAL\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [8]:
import pandas as pd
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from deepforest import CascadeForestRegressor
import numpy as np
np.int = int
np.bool = bool

In [9]:
def evaluate_all_combinations_cv_deepforest(X, y):
    results = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    for r in range(1, len(X.columns) + 1):
        for combo in itertools.combinations(X.columns, r):
            X_subset = X[list(combo)]
            model = CascadeForestRegressor(random_state=42, n_jobs=-1)
            scores = cross_val_score(model, X_subset, y, cv=kf, scoring='r2')
            mean_r2 = scores.mean()
            results.append({
                'num_features': r,
                'features': combo,
                'mean_r2': mean_r2
            })

    results_df = pd.DataFrame(results).sort_values(by='mean_r2', ascending=False).reset_index(drop=True)

    for i, row in results_df.iterrows():
        print(f"{i+1}. Features: {row['features']}, R²: {row['mean_r2']:.4f}")

    best = results_df.iloc[0]
    return best['features'], best['mean_r2'], results_df


In [10]:
df = pd.read_csv('dataset.csv')
X = df[['cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age']]
y = df['CCS']

# DeepForest로 조합 평가
best_combo, best_r2, all_results = evaluate_all_combinations_cv_deepforest(X, y)

print("\n📌 최고 조합:", best_combo)
print("📈 최고 평균 결정계수(R²):", best_r2)

[2025-07-16 06:00:31.818] Start to fit the model:
[2025-07-16 06:00:31.818] Fitting cascade layer = 0 
[2025-07-16 06:00:32.488] layer = 0  | Val MSE = 199.74430 | Elapsed = 0.670 s
[2025-07-16 06:00:32.488] Fitting cascade layer = 1 
[2025-07-16 06:00:33.255] layer = 1  | Val MSE = 149.06091 | Elapsed = 0.767 s
[2025-07-16 06:00:33.262] Fitting cascade layer = 2 
[2025-07-16 06:00:34.005] layer = 2  | Val MSE = 160.63274 | Elapsed = 0.744 s
[2025-07-16 06:00:34.005] Early stopping counter: 1 out of 2
[2025-07-16 06:00:34.009] Fitting cascade layer = 3 
[2025-07-16 06:00:34.756] layer = 3  | Val MSE = 165.25748 | Elapsed = 0.747 s
[2025-07-16 06:00:34.757] Early stopping counter: 2 out of 2
[2025-07-16 06:00:34.757] Handling early stopping
[2025-07-16 06:00:34.758] The optimal number of layers: 2
[2025-07-16 06:00:34.762] Start to evalute the model:
[2025-07-16 06:00:34.762] Evaluating cascade layer = 0 
[2025-07-16 06:00:34.888] Evaluating cascade layer = 1 
[2025-07-16 06:00:35.065] 